# Intro to Built-In Functions from `contrib.functions`


## Initial Setup

Lets first import the necessary modules and define the agents.

In [1]:
import os
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.functions import youtube_utils as yt
from autogen.agentchat.contrib.functions import file_utils as fu

In [2]:
config_list = [
    {
        "model": "gpt-4",
        "api_key": os.environ.get("OPENAI_API_KEY"),
    }
]

assistant = AssistantAgent(name="coder", llm_config={"config_list": config_list, "cache": None})
user = UserProxyAgent(
    name="user",
    code_execution_config={
        "work_dir": "/tmp",
    },
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and "TERMINATE" in x.get("content", ""),
)

## Simple Example

In [3]:
assistant.register_for_llm(description="Fetch transcript of a youtube video")(yt.get_youtube_transcript)
user.register_for_execution()(yt.get_youtube_transcript)

result = user.initiate_chat(
    assistant,
    message="Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c",
    summary_method="last_msg",
)

user (to coder):

Please summarize the video: https://www.youtube.com/watch?v=9iqn1HhFJ6c

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_uu7fVdiv87S8pLLRxFxYRGtV): get_youtube_transcript *****
Arguments: 
{
  "youtube_link": "https://www.youtube.com/watch?v=9iqn1HhFJ6c"
}
***************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_youtube_transcript...
requested package: youtube_transcript_api 0.6.0
found package youtube-transcript-api 0.6.0
user (to coder):

user (to coder):

***** Response from calling tool "call_uu7fVdiv87S8pLLRxFxYRGtV" *****
now ai is a great thing because AI will solve all the problems that we have today it will solve employment it will solve disease it will solve poverty but it will also create new problems the problem of fake news is goi

## Adavanced Example

Lets import multiple functions and use them accomplish more complex tasks.

In [4]:
# register multiple file reading functions
for foo in [
    fu.read_text_from_image,
    fu.read_text_from_pdf,
    fu.read_text_from_docx,
    fu.read_text_from_pptx,
    fu.read_text_from_xlsx,
    fu.read_text_from_audio,
]:
    foo_desc = foo.__doc__  # get doctring of the function
    assistant.register_for_llm(description=foo_desc)(foo)
    user.register_for_execution()(foo)

In [5]:
dummy_png = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg"
dummy_pdf = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
dummy_mp3 = "https://github.com/realpython/python-speech-recognition/raw/master/audio_files/harvard.wav"

result = user.initiate_chat(
    assistant,
    message=f"Please summarize the contents of the following files: {' '.join([dummy_png, dummy_pdf, dummy_mp3])}",
    summary_method="last_msg",
)

user (to coder):

Please summarize the contents of the following files: https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf https://github.com/realpython/python-speech-recognition/raw/master/audio_files/harvard.wav

--------------------------------------------------------------------------------
coder (to user):

***** Suggested tool Call (call_9EQ7lnpwFLxStOqmaITZVeMI): read_text_from_image *****
Arguments: 
{
  "file_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Captioned_image_dataset_examples.jpg/1024px-Captioned_image_dataset_examples.jpg"
}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION read_text_from_image...
requested package: pillow None
found package pi

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


user (to coder):

user (to coder):

***** Response from calling tool "call_9EQ7lnpwFLxStOqmaITZVeMI" *****
This flower has The flower Purple petals This flower has petals that are shown has white that are almost petals that are red with yellow petals with heart shaped yellow with many tips: yellow anther in with small green layers: L the center: receptacles: 8 J This bird is blue This bird is This is a bird bird with long with white on completely with a green black wings, tail the head.the black with a wing; a brown and white feathers are blue large blunt head and a red breast; the bill is and the belly is beak bill: short and black: 1 white. 3 Two people Lunch of rice A big rig truck in A group of men holding and beans with a parking lot traveling on snowboards are soup and juice: without a trailer: horses in the standing in the water; snow 8
**********************************************************************

------------------------------------------------------------------------